# Applied Data Science Capstone Week 5 The Battle of Neighborhoods

## A description of the problem and a discussion of the background

I like Ginza area in Japan.
So I would like to know neiborhoods that are similar to Ginza of Toronto and NYC.

## A description of the data and how it will be used to solve the problem.

Retribe venues data of from Foursquare API. Specialy restaurants and shops.
Clustering them.

## Data

In [74]:
import pandas as pd

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab


from botocore.client import Config
import ibm_boto3
import codecs

print('Libraries imported.')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [75]:
# The code was removed by Watson Studio for sharing.

In [76]:
# The code was removed by Watson Studio for sharing.

## The geograpical coordinate of Ginza .

In [77]:
# ginza_latitude = 35.667
# ginza_longitude = 139.767

## Clustering Venues 

I thought that it is better ways to know what neighboorhoods are similar to Ginza that they have similar tier and rating.
Because of quotas of Foursquare api, I saved responses and retrive data that I needed from them.

In [78]:
def get_object_from_cos(key_name):
    """
    Get utf-8-sig object from  IBM Cloud Object Storage
    """
    fileobject = credentials.get_object(Bucket=bucket_name ,Key=key_name)['Body']
    return fileobject

def get_json(key_name):
    """
    read from cos. and parse it as jdon.
    
    
    Parameters
    ----------
    key_names : string
        Key of IBM Cloud Object Storage

    Returns
    -------
    json_data : dict
        dict contain json read
    """
    file_object = get_object_from_cos(key_name)
    str_utf8 = codecs.decode(file_object.read(), encoding='utf-8-sig')
    json_data = json.loads(str_utf8)
    return json_data

def create_dataframe_of_venues(_key_names):
    """
     retrun Dataframe that contains venus.

    Parameters
    ----------
    _key_names : list of string
        list of key of IBM Cloud Object Storage

    Returns
    -------
    df_venues : pd.DataFrame
        DataFrame of venues
    """
    venues_list = []

    for key_name in _key_names:
        venues_json = get_json(key_name)
        for neiborhood_venus in venues_json["neighborhoods"]:
            for venue in neiborhood_venus["venues"]:
                venue_col = (neiborhood_venus["neighborhood"],venue["id"],venue["name"],venue["rating"],venue["tier"])
                venues_list.append( venue_col )
    df_venues = pd.DataFrame([item for item in venues_list])
    df_venues.columns = ['Neighborhood', 
                  'Venue Id', 
                  'Venue Nane', 
                  'Venue Rating', 
                  'Venue Tier']
    return df_venues


def read_neighborhood(key_name):
    """
     retrun list of neighborhood names

    Parameters
    ----------
    key_names : string
        key of IBM Cloud Object Storage

    Returns
    -------
    df_venues : pd.DataFrame
        DataFrame of venues
    """    
    json_neighborhoods =  get_json(key_name)
    return [neighborhood['name']  for neighborhood in json_neighborhoods['Neighborhoods']]

In [79]:
df_venues=create_dataframe_of_venues(key_names)
df_venues.tail(10)

,Neighborhood,Venue Id,Venue Nane,Venue Rating,Venue Tier
7806,Grange Park,4b61f9c1f964a520812e2ae3,Anh Dao,7.8,2
7807,Grange Park,4f57af17e4b01cdf1e8bfa45,Kanto,8.0,2
7808,Grange Park,4ad7e7acf964a520f40f21e3,Urban Herbivore,8.1,2
7809,Grange Park,54fce36c498eebc206762bcc,Homemade Ramen 蘭州拉麵,7.4,1
7810,Grange Park,4e78d6d5091a2a74e0523d83,Yummy Yummy Dumpling 口口香餃子館,7.4,1
7811,Grange Park,4adb5472f964a520fc2521e3,Asian Legend 味香村,7.9,2
7812,Grange Park,503fb59ce4b050691ba405a3,Gushi,7.7,2
7813,Grange Park,4b7716a9f964a520297c2ee3,Mother's Dumplings,7.9,1
7814,Grange Park,4ae348cef964a520dd9221e3,KOS,6.1,1
7815,Grange Park,50e6282de4b026ac78e0ee70,Pho Tien,5.8,2


In [80]:
df_data = df_venues.drop(columns=['Neighborhood','Venue Id','Venue Nane'],axis='columns')

min_max_scaler = preprocessing.MinMaxScaler()
np_venues = min_max_scaler.fit_transform(df_data.values)

for k in range(1,12):
    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans_result = kmeans.fit(np_venues)
    print("k={}".format(k))
    print(" kmeans_result.labels_.shape = "+str(kmeans_result.labels_.shape))
    print(" kmeans_result.labels_ = "+str(kmeans_result.labels_))
    print(" kmeans_result.inertia_ = "+str(kmeans_result.inertia_))


k=1
 kmeans_result.labels_.shape = (7816,)
 kmeans_result.labels_ = [0 0 0 ... 0 0 0]
 kmeans_result.inertia_ = 825.8176077117453
k=2
 kmeans_result.labels_.shape = (7816,)
 kmeans_result.labels_ = [1 0 0 ... 1 1 0]
 kmeans_result.inertia_ = 432.23126337061603
k=3
 kmeans_result.labels_.shape = (7816,)
 kmeans_result.labels_ = [0 1 2 ... 0 0 1]
 kmeans_result.inertia_ = 286.94444925375717
k=4
 kmeans_result.labels_.shape = (7816,)
 kmeans_result.labels_ = [3 0 2 ... 3 1 1]
 kmeans_result.inertia_ = 178.3844356147369
k=5
 kmeans_result.labels_.shape = (7816,)
 kmeans_result.labels_ = [2 1 3 ... 2 0 4]
 kmeans_result.inertia_ = 122.34535645096653
k=6
 kmeans_result.labels_.shape = (7816,)
 kmeans_result.labels_ = [0 1 5 ... 0 4 3]
 kmeans_result.inertia_ = 99.3999783482466
k=7
 kmeans_result.labels_.shape = (7816,)
 kmeans_result.labels_ = [1 2 6 ... 1 4 3]
 kmeans_result.inertia_ = 80.08036251900168
k=8
 kmeans_result.labels_.shape = (7816,)
 kmeans_result.labels_ = [3 1 0 ... 3 2 5]
 k

In [81]:
k_neighbors = 8

k_venues=5

kmeans = KMeans(n_clusters=k_venues, random_state=0)
kmeans_result = kmeans.fit(np_venues)

df_venues['Cluster Labels'] = kmeans_result.labels_
df_venues.head(10)

,Neighborhood,Venue Id,Venue Nane,Venue Rating,Venue Tier,Cluster Labels
0,Ginza,4bb5ea01941ad13a2aa21ee3,Grill Bon (新世界 グリル梵),9.0,1,2
1,Ginza,4b5d1a37f964a520c15229e3,Ku-ta (飯家くーた),8.9,2,1
2,Ginza,4c4859db1ddec9289235a032,Ginza Ukai-tei (銀座 うかい亭),8.4,4,3
3,Ginza,4c525974d797e21e9bb18f7d,Ginza Gyuan (銀座 牛庵),8.5,4,3
4,Ginza,4b766874f964a520594b2ee3,Sushizanmai Okunoin (すしざんまい 奥の院),8.1,2,1
5,Ginza,4c36fcf693db0f4764ec1e92,すしざんまい 新館,7.9,2,1
6,Ginza,4d259f871d9fa1cd61234109,Chika no Iki (地下の粋),8.3,2,1
7,Ginza,4b65376cf964a52048e92ae3,Nair's Restaurant (ナイルレストラン),8.7,2,1
8,Ginza,4b59bab9f964a520fc9428e3,Sushizanmai (すしざんまい 本店),8.9,2,1
9,Ginza,4d72073c311f6ea80175d5ea,Tsukiji Itadori Bekkan (築地虎杖 別館),9.0,2,1


## Toront's neigborhoods and Ginza.
Try to get neighboorhood in Toronto that is similar to Ginza.

In [82]:
# NYC_neighborhoods.json  toronto_neighborhoods.json
neighborfoods_t_g = read_neighborhood(toronto_neighborhoods_key)

neighborfoods_t_g = ['Ginza'] + neighborfoods_t_g

dict_toronto_ginza = {'Neighborhoods':neighborfoods_t_g}
# Add venues' in Ginza area to Toront neighborhood venues.
for i in range(k_venues):
    dict_toronto_ginza['venue {}'.format(i)]=[0 for j in range( len(neighborfoods_t_g) )]
                   
df_toronto_ginza = pd.DataFrame(dict_toronto_ginza)

for index, row in df_venues.iterrows():
    df_toronto_ginza.loc[df_toronto_ginza['Neighborhoods'] == row['Neighborhood'], 'venue {}'.format(row['Cluster Labels']  )] += 1
print(df_toronto_ginza.shape)
df_toronto_ginza.head()

(73, 6)


,Neighborhoods,venue 0,venue 1,venue 2,venue 3,venue 4
0,Ginza,0,46,13,11,1
1,Kensington Market,1,22,32,0,1
2,The Junction South,6,3,1,0,2
3,Forest Hill SE,3,1,0,0,2
4,Moore Park,0,0,0,0,0


In [83]:
# Drop a column
df_toronto_ginza=df_toronto_ginza.drop(columns=['Neighborhoods'],axis='columns')

kmeans = KMeans(n_clusters=k_neighbors, random_state = 0)
kmeans_result = kmeans.fit(df_toronto_ginza)
df_toronto_ginza['Neighborhoods'] = neighborfoods_t_g
df_toronto_ginza['Cluster'] = kmeans_result.labels_
fixed_columns = list(df_toronto_ginza.columns[-2:]) + list(df_toronto_ginza.columns[:-2])

df_toronto_ginza = df_toronto_ginza[fixed_columns]
df_toronto_ginza.head(10)

['Neighborhoods', 'Cluster', 'venue 0', 'venue 1', 'venue 2', 'venue 3', 'venue 4']


,Neighborhoods,Cluster,venue 0,venue 1,venue 2,venue 3,venue 4
0,Ginza,7,0,46,13,11,1
1,Kensington Market,5,1,22,32,0,1
2,The Junction South,0,6,3,1,0,2
3,Forest Hill SE,3,3,1,0,0,2
4,Moore Park,3,0,0,0,0,0
5,Rathnelly,3,3,1,0,0,2
6,The Beaches West,3,2,1,1,0,4
7,Trinity,6,2,17,9,5,5
8,Ryerson,4,21,18,13,4,20
9,Roncesvalles,3,1,3,0,1,1


In [84]:
# Show Gibza and neighborhood(s) of Toronto that is similar to Ginza.
df_toronto_ginza[df_toronto_ginza.Cluster == df_toronto_ginza.iloc[0].Cluster]

,Neighborhoods,Cluster,venue 0,venue 1,venue 2,venue 3,venue 4
0,Ginza,7,0,46,13,11,1


It seems there is no neighborhood that is similar to Ginza in Toronto.

## NYC's neighborhoods and Ginza.
Try to get neighboorhood in NYC that is similar to Ginza.

In [85]:
# NYC_neighborhoods.json 
neighborfoods_n_g = read_neighborhood(nyc_neighborhoods_key)

neighborfoods_n_g = ['Ginza'] + neighborfoods_n_g

dict_nyc_ginza = {'Neighborhoods':neighborfoods_n_g}
# Add venues' in Ginza area to NYC neighborhood venues.
for i in range(k_venues):
    dict_nyc_ginza['venue {}'.format(i)]=[0 for j in range( len(neighborfoods_n_g) )]
                   
df_nyc_ginza = pd.DataFrame(dict_nyc_ginza)

for index, row in df_venues.iterrows():
    df_nyc_ginza.loc[df_nyc_ginza['Neighborhoods'] == row['Neighborhood'], 'venue {}'.format(row['Cluster Labels']  )] += 1
print(df_nyc_ginza.shape)
df_nyc_ginza.head(20)

(307, 6)


,Neighborhoods,venue 0,venue 1,venue 2,venue 3,venue 4
0,Ginza,0,46,13,11,1
1,Wakefield,2,0,0,0,1
2,Co-op City,2,0,0,0,1
3,Eastchester,4,1,3,1,1
4,Fieldston,0,0,0,0,0
5,Riverdale,3,9,4,2,14
6,Kingsbridge,10,6,10,0,4
7,Marble Hill,4,1,3,1,1
8,Woodlawn,2,2,2,0,0
9,Norwood,7,0,3,0,2


In [86]:
# Drop a column.
df_nyc_ginza=df_nyc_ginza.drop(columns=['Neighborhoods'],axis='columns')

kmeans = KMeans(n_clusters=k_neighbors, random_state=0)
kmeans_result = kmeans.fit(df_nyc_ginza)
df_nyc_ginza['Neighborhoods'] = neighborfoods_n_g
df_nyc_ginza['Cluster'] = kmeans_result.labels_
fixed_columns = list(df_nyc_ginza.columns[-2:]) + list(df_nyc_ginza.columns[:-2])
print(str(fixed_columns))
df_nyc_ginza = df_nyc_ginza[fixed_columns]
df_nyc_ginza.head(10)

['Neighborhoods', 'Cluster', 'venue 0', 'venue 1', 'venue 2', 'venue 3', 'venue 4']


,Neighborhoods,Cluster,venue 0,venue 1,venue 2,venue 3,venue 4
0,Ginza,3,0,46,13,11,1
1,Wakefield,0,2,0,0,0,1
2,Co-op City,0,2,0,0,0,1
3,Eastchester,6,4,1,3,1,1
4,Fieldston,0,0,0,0,0,0
5,Riverdale,2,3,9,4,2,14
6,Kingsbridge,2,10,6,10,0,4
7,Marble Hill,6,4,1,3,1,1
8,Woodlawn,0,2,2,2,0,0
9,Norwood,6,7,0,3,0,2


In [87]:
# Show Gibza and neighborhood(s) of NYC that is similar to Ginza.
df_nyc_ginza[df_nyc_ginza.Cluster == df_nyc_ginza.iloc[0].Cluster]

,Neighborhoods,Cluster,venue 0,venue 1,venue 2,venue 3,venue 4
0,Ginza,3,0,46,13,11,1
97,North Side,3,0,46,33,12,6
115,Midtown,3,0,39,35,22,1
116,Murray Hill,3,4,58,25,22,20
119,East Village,3,0,51,27,17,0
122,Little Italy,3,0,48,33,17,0
129,Financial District,3,0,36,46,12,0
181,Murray Hill,3,4,58,25,22,20
249,Noho,3,0,57,29,14,0
251,Midtown South,3,0,56,26,16,0


9 neighborhoods that are similar to Ginza.